In [1]:
import torch
print(torch.__version__)

2.1.0


In [2]:
import cv2
import numpy as np
from mtcnn import MTCNN
from deep_sort_realtime.deepsort_tracker import DeepSort
import face_recognition
import os
import numpy as np
import joblib

In [3]:
# Initialize DeepSORT tracker
tracker = DeepSort(max_age=10)

In [4]:
# Load pre-trained MTCNN model for face detection
mtcnn = MTCNN()

In [5]:
# Open video file
video_path = "Class Room Entrance.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [6]:
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for mp4 format
output_path = "output_video.mp4"
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

In [7]:
frame_id = 0
skip_frames = 2  # Skip the frames

In [8]:
# Size threshold for faces (adjust as needed)
min_face_size = 10000  # For example, minimum area of 10000 pixels

In [9]:
# Dictionary to track whether the face image and embedding have been saved for each ID
saved_data = {}

In [10]:
# Create a directory to save face images
os.makedirs("faces(V3)", exist_ok=True)

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # Skip frames until reaching the nth frame
    if frame_id <= skip_frames:
        continue

    # Detect faces using MTCNN
    faces = mtcnn.detect_faces(frame)
    bbs = [(face['box'], face['confidence']) for face in faces] #, face['keypoints']
    print(bbs)
    # Update tracker with the detected faces
    tracks = tracker.update_tracks(bbs, frame=frame)

    # Draw bounding boxes and IDs on the frame
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id

        # Check if data for this ID has already been saved
        if track_id in saved_data:
            continue

        ltrb = track.to_ltrb()

        # Check face size
        face_area = (ltrb[2] - ltrb[0]) * (ltrb[3] - ltrb[1])
        if face_area < min_face_size:
            continue  # Skip small faces

        # Draw bounding box
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 2)

        # Get facial landmarks using face_recognition library
        face_locations = [(int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0]))]  # (top, right, bottom, left)
        landmarks = face_recognition.face_landmarks(frame, face_locations)

        if landmarks:
            # Get face embedding
            face_encoding = face_recognition.face_encodings(frame, face_locations)[0]

            # Check if the face embedding is not empty before saving
            if len(face_encoding) > 0:
                # Compare with existing embeddings
                match = False
                for saved_id, saved_info in saved_data.items():
                    saved_embedding = saved_info["embedding"]
                    # Compare the face embeddings using face_recognition library
                    results = face_recognition.compare_faces([saved_embedding], face_encoding)

                    # Check if there is a match
                    if any(results):
                        match = True
                        break

                # If no match, save the face embedding
                if not match:
                    # Save the face embedding
                    saved_data[track_id] = {
                        "embedding": face_encoding,
                        "image_path": f"faces(V3)/{track_id}_frame{frame_id}.png"
                    }

                    # Get the face bounding box coordinates
                    top, right, bottom, left = int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0])

                    # Check if the bounding box coordinates are within the frame boundaries
                    if 0 <= top < frame.shape[0] and 0 <= left < frame.shape[1] and top < bottom and left < right:
                        # Capture the face image
                        face_image = frame[top:bottom, left:right]

                        # Check if the face image is not empty before saving
                        if not face_image.size == 0:
                            # Save the face image
                            cv2.imwrite(saved_data[track_id]["image_path"], face_image)

                            # Draw label
                            label = f"ID: {track_id}"
                            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the frame with bounding boxes to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 222ms/step
[([595, 64, 80, 110], 0.9981001019477844), ([345, 49, 30, 38], 0.9841081500053406), ([481, 58, 28, 38], 0.9772443771362305), ([582, 54, 31, 42], 0.9156140685081482)]
1/1 [==============================] - 0s 41ms/step
[([598, 57, 79, 112], 0.9980218410491943), ([340, 49, 31, 39], 0.9974222779273987), ([477, 57, 27, 38], 0.9884970784187317), ([583, 57, 31, 40], 0.9821586608886719), ([201, 59, 40, 51], 0.8968363404273987)]
1/1 [==============================] - 0s 39ms/step
[([597, 57, 80, 112], 0.99806809425354), ([477, 57, 27, 38], 0.9923795461654663), ([583, 57, 31, 40], 0.9833961129188538), ([341, 49, 30, 38], 0.9639467597007751), ([201, 59, 40, 51], 0.8969258069992065)]
1/1 [==============================] - 0s 46ms/step
[([596, 56, 78, 107], 0.9965737462043762), ([337, 49, 30, 38], 0.9893863201141357), ([472, 55, 28, 39], 0.9872264862060547), ([583, 55, 31, 42], 0.9759958982467651), ([199, 62, 44, 60], 0.9166142344474792)]
1/1 [=

c:\Users\SAnuv\anaconda3\envs\ahead\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\SAnuv\anaconda3\envs\ahead\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\SAnuv\anaconda3\envs\ahead\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\SAnuv\anaconda3\envs\ahead\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 

In [ ]:
saved_data

In [ ]:
# Load the age prediction model
loaded_age_model = joblib.load('age_model.pkl')

# Load the gender classification model
loaded_gender_model = joblib.load('gender_model.pkl')

# Lists to store predicted ages and genders
age_array = []
gender_array = []

# Iterate through saved_data for age and gender prediction
for track_id, info in saved_data.items():
    # Get the face embedding
    face_embedding = info["embedding"]

    # Reshape the face embedding to match the input shape expected by the models
    face_embedding_reshaped = face_embedding.reshape(1, -1)
    print(face_embedding_reshaped)
    # Predict age
    predicted_age = loaded_age_model.predict(face_embedding_reshaped)[0]
    age_array.append(predicted_age)

    # Predict gender
    predicted_gender = loaded_gender_model.predict(face_embedding_reshaped)[0]
    gender_array.append(predicted_gender)

# Convert age_array and gender_array to numpy arrays
age_array = np.array(age_array)
gender_array = np.array(gender_array)

# Print or use the predicted age and gender arrays as needed
print("Predicted Ages:", age_array)
print("Predicted Genders:", gender_array)
